In [1]:
!pip install -q transformers datasets accelerate torchvision
!pip install --upgrade accelerate
!pip install --upgrade transformers
!pip install -q sentencepiece
!pip install -q jiwer
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.

In [1]:
!pip install -q transformers
!pip install -q sentencepiece
!pip install -q jiwer
!pip install -q datasets
!pip install -q evaluate
!pip install -q -U accelerate

!pip install -q matplotlib
!pip install -q protobuf==3.20.1
!pip install -q tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.4 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine =

In [18]:
import os
from PIL import Image
import numpy as np
from datasets import Dataset
import pandas as pd
import torch
from transformers import TrOCRProcessor, default_data_collator
import evaluate
import numpy as np
import pandas as pd
import glob as glob
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

 
 
from PIL import Image
from zipfile import ZipFile
from tqdm.notebook import tqdm
from dataclasses import dataclass
dir_path = '/kaggle/input/groundtruthtext'
train_path = dir_path + "/" + 'train/gt_text'

# Prepare Dataset

In [23]:
train_df = pd.read_csv(dir_path + "/" + "train.csv")
train_df['image'] = train_df['image'].apply(lambda x: dir_path + "/" + x)
test_df = pd.read_csv(dir_path + "/" + "test.csv")
test_df['image'] = test_df['image'].apply(lambda x: dir_path + "/" + x)

train_df.head()
merge_df = pd.concat([train_df.iloc[:4000], train_df.iloc[10000:12500], train_df.iloc[17500:19000]])
dataset = Dataset.from_pandas(merge_df)

In [24]:
import torch

def preprocess(example):
    image = Image.open(example["image"]).convert("RGB")

    # Process image
    pixel_values = processor(images=image, return_tensors="pt").pixel_values[0]#.half()
    example["pixel_values"] = pixel_values
    # example["pixel_values"] = pixel_values.tolist()  # Convert tensor to list

    # Ensure text is not None
    if example.get("text") is None:
        example["text"] = ""

    # Tokenize text
    labels = processor.tokenizer(example["text"], return_tensors="pt", padding="max_length", max_length=64).input_ids[0]
    labels = [label if label != processor.tokenizer.pad_token_id else -100 for label in labels]
    example["labels"] = labels

    return example

In [42]:
# import shutil

# def zip_directory(dir_path, output_path):
#     shutil.make_archive(output_path, 'zip', dir_path)
#     print(f"Directory '{dir_path}' zipped successfully to '{output_path}.zip'")

# # Example usage
# dir_to_zip = "/kaggle/working/trocr-finetuned/output"
# output_zip_file = "/kaggle/working/trocr-finetuned/out"
# zip_directory(dir_to_zip, output_zip_file)

Directory '/kaggle/working/trocr-finetuned/output' zipped successfully to '/kaggle/working/trocr-finetuned/out.zip'


In [21]:
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten", use_fast=True)

In [25]:
processed_dataset = dataset.map(preprocess)
# processed_dataset.save_to_disk(f"/kaggle/working/final")

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [10]:
# from datasets import load_from_disk, concatenate_datasets
# n_parts = 10
# parts = [load_from_disk(f"/kaggle/working/processed_part_{i}") for i in range(n_parts)]
# final_dataset = concatenate_datasets(parts)

# Train Model

# Method 1

In [4]:
@dataclass(frozen=True)
class TrainingConfig:
    BATCH_SIZE:    int = 48
    EPOCHS:        int = 1
    LEARNING_RATE: float = 0.00005
 
@dataclass(frozen=True)
class DatasetConfig:
    DATA_ROOT:     str = '/kaggle/input'
 
@dataclass(frozen=True)
class ModelConfig:
    MODEL_NAME: str = 'microsoft/trocr-base-handwritten'


In [5]:
train_df = pd.read_csv(os.path.join(DatasetConfig.DATA_ROOT, 'groundtruthtext/train.csv'))
test_df = pd.read_csv(os.path.join(DatasetConfig.DATA_ROOT, 'groundtruthtext/test.csv'))

In [6]:
class CustomOCRDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=64):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length
 
 
    def __len__(self):
        return len(self.df)
 
 
    def __getitem__(self, idx):
        # The image file name.
        file_name = self.df['image'][idx]
        # The text (label).
        text = self.df['text'][idx]
        # Read the image, apply augmentations, and get the transformed pixels.
        image = Image.open(self.root_dir + file_name).convert('RGB')
        # image = train_transforms(image)
        pixel_values = self.processor(image, return_tensors='pt').pixel_values
        # Pass the text through the tokenizer and get the labels,
        # i.e. tokenized labels.
        labels = self.processor.tokenizer(
            text,
            padding='max_length',
            max_length=self.max_target_length
        ).input_ids
        # We are using -100 as the padding token.
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]
        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

In [7]:
processor = TrOCRProcessor.from_pretrained(ModelConfig.MODEL_NAME)
train_dataset = CustomOCRDataset(
    root_dir=os.path.join(DatasetConfig.DATA_ROOT, 'groundtruthtext/'),
    df=train_df,
    processor=processor
)
valid_dataset = CustomOCRDataset(
    root_dir=os.path.join(DatasetConfig.DATA_ROOT, 'groundtruthtext/'),
    df=test_df,
    processor=processor
)

preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [8]:
encoding = train_dataset[0]
for k,v in encoding.items():
    print(k, v.shape)

pixel_values torch.Size([3, 384, 384])
labels torch.Size([64])


In [9]:
# # load dataset
# from datasets import load_from_disk
# from datasets import load_dataset
# # processed_dataset = load_from_disk("/kaggle/input/processedgt/final/dataset_info.json")
# # len(processed_dataset)
# processed_dataset = load_dataset('json', data_files='/kaggle/input/processedgt/final/dataset_info.json', streaming=True)
# len(processed_dataset)

In [10]:
from transformers import VisionEncoderDecoderModel, Seq2SeqTrainer, Seq2SeqTrainingArguments, TrainerCallback

# Load model
# processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten", use_fast=True)
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")
model.to("cuda")
print(model)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

config.json:   0%|          | 0.00/4.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (i

In [6]:
# from transformers import DataCollatorForSeq2Seq

# data_collator = DataCollatorForSeq2Seq(
#     processor.tokenizer, 
#     model=None,  # Set later after model is loaded
#     padding=True, 
#     return_tensors="pt"
# )

In [11]:
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# Set Correct vocab size.
model.config.vocab_size = model.config.decoder.vocab_size
model.config.eos_token_id = processor.tokenizer.sep_token_id
 
 
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [12]:
optimizer = torch.optim.AdamW(
    model.parameters(), lr=0.00005, weight_decay=0.0005
)

In [13]:
# import evaluate
cer_metric = evaluate.load('cer')
 
 
def compute_cer(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
 
 
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)
 
 
    cer = cer_metric.compute(predictions=pred_str, references=label_str)
 
 
    return {"cer": cer}

In [14]:
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    eval_strategy='epoch',
    per_device_train_batch_size=TrainingConfig.BATCH_SIZE,
    per_device_eval_batch_size=TrainingConfig.BATCH_SIZE,
    fp16=True,
    output_dir='seq2seq_model/',
    logging_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=5,
    report_to='tensorboard',
    num_train_epochs=TrainingConfig.EPOCHS
)
# training_args = Seq2SeqTrainingArguments(
#     output_dir="./trocr-finetuned",
#     per_device_train_batch_size=48,
#     # evaluation_strategy="epoch",
#     save_steps=500,
#     logging_steps=50,
#     num_train_epochs=3,
#     fp16=True,
#     predict_with_generate=True,
#     report_to="none",
# )

In [15]:
class PrintLogsCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            print(logs)

In [16]:
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator(return_tensors="pt")
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=processed_dataset,
#     compute_metrics=compute_cer,
#     eval_dataset=processed_dataset.select(range(20)),  # Optional
#     data_collator=default_data_collator,
#     tokenizer=processor.tokenizer,
# )
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=processor.tokenizer,
    args=training_args,
    compute_metrics=compute_cer,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator
)

/tmp/ipykernel_35/3932816936.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [17]:
# model.config.decoder_start_token_id = processor.tokenizer.cls_token_id

# trainer.add_callback(PrintLogsCallback())
res = trainer.train()

AttributeError: 'CustomOCRDataset' object has no attribute '_data'

# Method 2

In [26]:
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten", use_fast=True)
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")
model.to("cuda")
print(model)

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (i

In [27]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    processor.tokenizer, 
    model=model,  # Set later after model is loaded
    padding=True, 
    return_tensors="pt"
)

In [28]:
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# Set Correct vocab size.
model.config.vocab_size = model.config.decoder.vocab_size
model.config.eos_token_id = processor.tokenizer.sep_token_id
 
 
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [29]:
cer_metric = evaluate.load('cer')
 
 
def compute_cer(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
 
 
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)
 
 
    cer = cer_metric.compute(predictions=pred_str, references=label_str)
 
 
    return {"cer": cer}


In [61]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./trocr-finetuned",
    per_device_train_batch_size=2,
    eval_strategy="steps",
    save_steps=100,
    logging_steps=20,
    eval_steps = 20,
    num_train_epochs=3,
    fp16=True,
    predict_with_generate=True,
    report_to="none",
    gradient_accumulation_steps=24
)

In [62]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    compute_metrics=compute_cer,
    eval_dataset=processed_dataset.select(range(50)),  # Optional
    data_collator=default_data_collator,
    tokenizer=processor.tokenizer
)

/tmp/ipykernel_35/798788004.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [64]:
optimizer = torch.optim.AdamW(
    model.parameters(), lr=0.00005, weight_decay=0.0005
)

In [ ]:
res = trainer.train()

Step,Training Loss,Validation Loss,Cer
20,1.860400,2.423681,0.910690
40,2.566500,2.128994,1.200271
60,2.499800,1.792621,1.453315
80,2.257000,1.689292,0.928281
100,1.972000,1.594626,1.110961
120,1.895100,1.315072,0.906631
140,1.862000,1.108550,0.745602
160,1.774000,1.148231,0.920162
180,1.541700,1.056035,0.768606
200,1.312600,0.953724,0.947226


In [41]:
model.save_pretrained("./trocr-finetuned/output")
processor.save_pretrained("./trocr-finetuned/output")

[]

In [ ]:
# processor = TrOCRProcessor.from_pretrained("./trocr-finetuned")
# model = VisionEncoderDecoderModel.from_pretrained("./trocr-finetuned")

# # Move model to GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# Load and preprocess the image


# Inference

In [45]:
# folder_path = train_path + "/" + "MIT_1"
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# for file in os.listdir(folder_path):        
    
#     image_path = folder_path + "/" + file
#     image = Image.open(image_path).convert("RGB")
#     pixel_values = processor(images=image, return_tensors="pt").pixel_values.to(device).half()
    
#     # Generate output
#     generated_ids = model.generate(pixel_values)
    
#     # Decode generated token ids to string
#     generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
#     print("Predicted Text:", generated_text)

Predicted Text: Indian-
Predicted Text: 12271975
Predicted Text: Lhhak Bak
Predicted Text: HNo 23 23 Ch Marg,,alla,ad,ada,ad-6449
Predicted Text: 9 atan Lan Lad
Predicted Text: 06171720
Predicted Text: H...No 13 B Circle Circle Circle, Circle,, Circle Circle English,,hing,,ig,,ugu
Predicted Text: Female
Predicted Text: A
Predicted Text: 10 athab Chal P
Predicted Text: PostGrad
Predicted Text: Mar English English Beng,ugu,ugu Hindi Hindi Hindi
Predicted Text: 35000000000000000..0..
Predicted Text: I, -3638 -38382838
Predicted Text: Mar
Predicted Text: 9310549969
Predicted Text: 97806161628
Predicted Text: Dehman
Predicted Text: Daylashishi
Predicted Text: Kik B -99999940644


In [46]:
# processor1 = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten", use_fast=True)
# model1 = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")
# model1.to("cuda")
# folder_path = train_path + "/" + "MIT_1"
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# for file in os.listdir(folder_path):        
    
#     image_path = folder_path + "/" + file
#     image = Image.open(image_path).convert("RGB")
#     pixel_values = processor1(images=image, return_tensors="pt").pixel_values.to(device).half()
    
#     # Generate output
#     generated_ids = model1.generate(pixel_values)
    
#     # Decode generated token ids to string
#     generated_text = processor1.batch_decode(generated_ids, skip_special_tokens=True)[0]
#     print("Predicted Text:", generated_text)

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

Predicted Text: Indian
Predicted Text: 12I 27(1975
Predicted Text: Laksh Bakshi
Predicted Text: H.NO. 23 , Chaudry Path , Nadjad - 5594 07
Predicted Text: a year's at Denian Lod
Predicted Text: ob ( 17 ( 2023
Predicted Text: # 13 3 , Bala circle , Bogggigan
Predicted Text: Female
Predicted Text: a t
Predicted Text: 10 years , at Chaka ) PLC
Predicted Text: Post- Graduate
Predicted Text: Marathi , English , Delugu , Hindi
Predicted Text: 35808600000
Predicted Text: Janaki Handa - 8634823848
Predicted Text: married
Predicted Text: 9351 045691
Predicted Text: 978 7612803 .
Predicted Text: Dehradun .
Predicted Text: Dayita Bakshi
Predicted Text: Kritika Brar - 7099406444


In [57]:
# processor1 = TrOCRProcessor.from_pretrained("microsoft/trocr-large-handwritten", use_fast=True)
# model1 = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-large-handwritten")
# model1.to("cuda")
folder_path = train_path + "/" + "MIT_1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for file in os.listdir(folder_path):        
    
    image_path = folder_path + "/" + file
    image = Image.open(image_path).convert("RGB")
    pixel_values = processor(images=image, return_tensors="pt").pixel_values.to(device).half()
    
    # Generate output
    generated_ids = model.generate(pixel_values)
    
    # Decode generated token ids to string
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print("Predicted Text:", generated_text)

Predicted Text: Indian
Predicted Text: 1212112121212212
Predicted Text: Lhakhi
Predicted Text: H... Che,,, Chow,, -
Predicted Text: 99 999949996 Ltd Ltd Ltd
Predicted Text: 06060660606230306062020230620066230666606630606260623620060616066203060306036616060603
Predicted Text: H33.. B B B,, B
Predicted Text: Female
Predicted Text: AAA
Predicted Text: 101010110101101010 1010101110104 P1010301010161010101LC
Predicted Text: Post/ G
Predicted Text: Mar,ati,,, English,,ali, Hindi,, Hindi Hindi, Hindi
Predicted Text: 3000000
Predicted Text: I H H -3838
Predicted Text: Mar
Predicted Text: 9300561
Predicted Text: 9181820
Predicted Text: DeDeDeunun
Predicted Text: DDitahii
Predicted Text: Kalitaarar -4949


In [59]:
cer_metric.compute(predictions='1212112121', references='12/27/1975')

0.7

In [60]:
len('06060660606230306062020230620066230666606630606260623620060616066203060306036616060603')

86